In [73]:
# -*- coding: utf-8 -*-
from __future__ import division
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from tqdm import tqdm

In [74]:
conv = Pipeline([
        ('vect', CountVectorizer(binary=True)),
        ('tfidf', TfidfTransformer()),
    ])

In [75]:
import csv
import codecs

In [76]:
train_data = []
train_target = []
test_data = []
test_target = []
with codecs.open('../data/interim/clear_train.csv', 'r') as train_file:
    data_reader = csv.reader(train_file)
    for row in tqdm(data_reader):
        train_data.append(row[0])
        train_target.append(float(row[1]))
with codecs.open('../data/interim/clear_test.csv', 'r') as test_file:
    data_reader = csv.reader(test_file)
    for row in tqdm(data_reader):
        test_data.append(row[0])
        test_target.append(float(row[1]))

3893it [00:00, 106450.39it/s]
4573it [00:00, 112174.19it/s]


In [77]:
train_tfidf = conv.fit_transform(train_data)
test_tfidf = conv.transform(test_data)
print train_tfidf.shape
print test_tfidf.shape

(3893, 13722)
(4573, 13722)


In [78]:
import pandas as pd

In [79]:
my_cols = ['word', 'pos', 'lemma', 'senti', 'source', 'tes']
vocab = pd.read_csv('../data/dict/dict.csv', names=my_cols)

In [80]:
#vocab.head()

In [81]:
import re
from stop_words import get_stop_words
stop_words = get_stop_words('ru')
#print stop_words
import numpy as np

In [82]:
lens = []
for line in codecs.open('../data/dict/AFINN-111-ru.txt', 'r'):
    lens.append(len(line.split()))
from collections import Counter
#Counter(lens)

In [83]:
mean_col = []
plus_col = []
minus_col = []
multi_col = []
with codecs.open('../data/interim/clear_train.csv', 'r') as train_file:
    data_reader = csv.reader(train_file)
    i = 0
    for idx, row in tqdm(enumerate(data_reader)):
        i += 1
        text = row[0]
        text1 = text.decode('utf-8')
        #print text
        mean_senti = 0
        plus_senti = 0
        minus_senti = 0
        multi_senti = 0
        for idx1, word in enumerate(text1.split()):
            #print word
            if (word not in stop_words) and (len(word) > 2):
                #print word
                res = vocab.loc[vocab['lemma'].str.contains(text.split()[idx1])]['senti']
                for senti in res:
                    #print senti.split()
                    if senti.split()[0] == 'positive':
                        plus_senti += 1
                    if senti.split()[0] == 'negative':
                        minus_senti += 1
                    if senti.split()[0] == 'positive/negative':
                        multi_senti += 1
        if (plus_senti > 0) or (minus_senti > 0):
            mean_senti = (plus_senti - minus_senti) / (plus_senti + minus_senti)
        #print mean_senti, plus_senti, minus_senti, multi_senti
        mean_col.append(mean_senti)
        plus_col.append(plus_senti)
        minus_col.append(minus_senti)
        multi_col.append(multi_senti)

3893it [18:36,  4.14it/s]


In [84]:
print len(mean_col)
a = np.array(mean_col)
a.shape
mean_col1 = np.reshape(a, len(mean_col), -1)
mean_col1.shape

3893


(3893,)

In [85]:
print len(plus_col)
a = np.array(plus_col)
plus_col1 = np.reshape(a, len(plus_col), -1)
plus_col1.shape

3893


(3893,)

In [86]:
print len(minus_col)
a = np.array(minus_col)
minus_col1 = np.reshape(a, len(minus_col), -1)
minus_col1.shape

3893


(3893,)

In [87]:
print len(multi_col)
print multi_col[0]

3893
3


In [88]:
a = np.array(multi_col)

In [89]:
print a.shape
multi_col1 = np.reshape(a, len(multi_col), -1)
multi_col1.shape

(3893,)


(3893,)

In [90]:
train_df = pd.DataFrame(train_tfidf.toarray())

In [91]:
train_df.insert(len(train_df),'mean', mean_col)

In [92]:
train_df.insert(len(train_df),'plus', plus_col)
train_df.insert(len(train_df),'minus', minus_col)
train_df.insert(len(train_df),'multi', multi_col)

In [93]:
train_tfidf = train_df.as_matrix()

In [94]:
train_tfidf.shape

(3893, 13726)

In [95]:
mean_col = []
plus_col = []
minus_col = []
multi_col = []
with codecs.open('../data/interim/clear_test.csv', 'r') as test_file:
    data_reader = csv.reader(test_file)
    i = 0
    for idx, row in tqdm(enumerate(data_reader)):
        i += 1
        text = row[0]
        text1 = text.decode('utf-8')
        #print text
        mean_senti = 0
        plus_senti = 0
        minus_senti = 0
        multi_senti = 0
        for idx1, word in enumerate(text1.split()):
            #print word
            if (word not in stop_words) and (len(word) > 2):
                #print word
                res = vocab.loc[vocab['lemma'].str.contains(text.split()[idx1])]['senti']
                for senti in res:
                    #print senti.split()
                    if senti.split()[0] == 'positive':
                        plus_senti += 1
                    if senti.split()[0] == 'negative':
                        minus_senti += 1
                    if senti.split()[0] == 'positive/negative':
                        multi_senti += 1
        if (plus_senti > 0) or (minus_senti > 0):
            mean_senti = (plus_senti - minus_senti) / (plus_senti + minus_senti)
        #print mean_senti, plus_senti, minus_senti, multi_senti
        mean_col.append(mean_senti)
        plus_col.append(plus_senti)
        minus_col.append(minus_senti)
        multi_col.append(multi_senti)

4573it [23:04,  2.19it/s]


In [96]:
test_df = pd.DataFrame(test_tfidf.toarray())
test_df.insert(len(test_df),'mean', mean_col)
test_df.insert(len(test_df),'plus', plus_col)
test_df.insert(len(test_df),'minus', minus_col)
test_df.insert(len(test_df),'multi', multi_col)
test_tfidf = test_df.as_matrix()
test_tfidf.shape

(4573, 13726)

In [97]:
from sklearn.datasets import dump_svmlight_file

In [98]:
dump_svmlight_file(train_tfidf, train_target, '../data/libsvm/train_tfidf.libsvm')
dump_svmlight_file(test_tfidf, test_target, '../data/libsvm/test_tfidf.libsvm')